In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
data = pd.read_csv('Google-Playstore-32K.csv', delimiter=',')
data.dataframeName = 'Google-Playstore-32K.csv'
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')


There are 32000 rows and 11 columns


In [3]:
data.head(5)

,App Name,Category,Rating,Reviews,Installs,Size,Price,Content Rating,Last Updated,Minimum Version,Latest Version
0,DoorDash - Food Delivery,FOOD_AND_DRINK,4.548561573,305034.0,"5,000,000+",Varies with device,0,Everyone,"March 29, 2019",Varies with device,Varies with device
1,TripAdvisor Hotels Flights Restaurants Attract...,TRAVEL_AND_LOCAL,4.400671482,1207922.0,"100,000,000+",Varies with device,0,Everyone,"March 29, 2019",Varies with device,Varies with device
2,Peapod,SHOPPING,3.656329393,1967.0,"100,000+",1.4M,0,Everyone,"September 20, 2018",5.0 and up,2.2.0
3,foodpanda - Local Food Delivery,FOOD_AND_DRINK,4.107232571,389154.0,"10,000,000+",16M,0,Everyone,"March 22, 2019",4.2 and up,4.18.2
4,My CookBook Pro (Ad Free),FOOD_AND_DRINK,4.647752285,2291.0,"10,000+",Varies with device,$5.99,Everyone,"April 1, 2019",Varies with device,Varies with device


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 11 columns):
App Name           32000 non-null object
Category           32000 non-null object
Rating             32000 non-null object
Reviews            31999 non-null float64
Installs           32000 non-null object
Size               32000 non-null object
Price              32000 non-null object
Content Rating     32000 non-null object
Last Updated       32000 non-null object
Minimum Version    32000 non-null object
Latest Version     31999 non-null object
dtypes: float64(1), object(10)
memory usage: 2.7+ MB


In [5]:
#detect null cols and null rate
nulls = [i for i in data.isna().any().index if data.isna().any()[i]==True]
rates = []
counts = []
for i in nulls:    
    rates.append((data[i].isna().sum()/data.shape[0])*100)
    counts.append(data[i].isna().sum())
null_df = pd.DataFrame.from_dict({"Col":nulls,"Count":counts,"Null_Rates":rates})
null_df

,Col,Count,Null_Rates
0,Reviews,1,0.003125
1,Latest Version,1,0.003125


In [6]:
#delete Type,Content Rating, Current Ver, Android Ver null values row
df_train = data.copy()
for i in ['Reviews','Latest Version']:
    df_train = df_train.drop(df_train.loc[df_train[i].isnull()].index,0)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31998 entries, 0 to 31999
Data columns (total 11 columns):
App Name           31998 non-null object
Category           31998 non-null object
Rating             31998 non-null object
Reviews            31998 non-null float64
Installs           31998 non-null object
Size               31998 non-null object
Price              31998 non-null object
Content Rating     31998 non-null object
Last Updated       31998 non-null object
Minimum Version    31998 non-null object
Latest Version     31998 non-null object
dtypes: float64(1), object(10)
memory usage: 2.9+ MB


In [7]:
df_train['Rating'].describe()

count     31998
unique    25433
top           5
freq       1014
Name: Rating, dtype: object

In [8]:
df_train['Rating'].replace('Lessons', np.nan, inplace=True)
df_train['Rating'].replace('GAME_STRATEGY', np.nan, inplace=True)
df_train['Rating'].replace('NEWS_AND_MAGAZINES', np.nan, inplace=True)

for i in ['Rating']:
    df_train = df_train.drop(df_train.loc[df_train[i].isnull()].index,0)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31996 entries, 0 to 31999
Data columns (total 11 columns):
App Name           31996 non-null object
Category           31996 non-null object
Rating             31996 non-null object
Reviews            31996 non-null float64
Installs           31996 non-null object
Size               31996 non-null object
Price              31996 non-null object
Content Rating     31996 non-null object
Last Updated       31996 non-null object
Minimum Version    31996 non-null object
Latest Version     31996 non-null object
dtypes: float64(1), object(10)
memory usage: 2.9+ MB


In [9]:
df_train['Rating'].astype(float).describe()

count    31996.000000
mean         4.271225
std          0.499642
min          1.000000
25%          4.090909
50%          4.375000
75%          4.583333
max          5.000000
Name: Rating, dtype: float64

In [10]:
df_train.Category.unique()

array(['FOOD_AND_DRINK', 'TRAVEL_AND_LOCAL', 'SHOPPING', 'LIFESTYLE',
       'GAME_ACTION', 'GAME_CASUAL', 'GAME_ROLE_PLAYING', 'GAME_PUZZLE',
       'GAME_RACING', 'GAME_ADVENTURE', 'GAME_ARCADE', 'GAME_STRATEGY',
       'GAME_SPORTS', 'GAME_SIMULATION', 'GAME_MUSIC', 'MUSIC_AND_AUDIO',
       'FINANCE', 'EVENTS', 'ENTERTAINMENT', 'EDUCATION',
       'GAME_EDUCATIONAL', 'BOOKS_AND_REFERENCE', 'NEWS_AND_MAGAZINES',
       'PHOTOGRAPHY', 'VIDEO_PLAYERS', 'GAME_WORD', 'ART_AND_DESIGN',
       'GAME_TRIVIA', 'GAME_BOARD', 'BUSINESS', 'PRODUCTIVITY',
       'COMMUNICATION', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'SOCIAL',
       'BEAUTY', 'GAME_CASINO', 'MAPS_AND_NAVIGATION', 'PERSONALIZATION',
       'GAME_CARD', 'TOOLS', 'SPORTS', 'AUTO_AND_VEHICLES',
       'LIBRARIES_AND_DEMO', 'COMICS', 'PARENTING', 'DATING', 'WEATHER',
       'MEDICAL'], dtype=object)

In [145]:
# convert to categorical Categority by using one hot tecnique 
df_dummy = df_train.copy()
df_dummy.Category = pd.Categorical(df_dummy.Category)

x = df_dummy[['Category']]
del df_dummy['Category']

dummies = pd.get_dummies(x, prefix = 'Category')
df_dummy = pd.concat([df_dummy,dummies], axis=1)
df_dummy.head()

,App Name,Rating,Reviews,Installs,Size,Price,Content Rating,Last Updated,Minimum Version,Latest Version,...,Category_PERSONALIZATION,Category_PHOTOGRAPHY,Category_PRODUCTIVITY,Category_SHOPPING,Category_SOCIAL,Category_SPORTS,Category_TOOLS,Category_TRAVEL_AND_LOCAL,Category_VIDEO_PLAYERS,Category_WEATHER
0,DoorDash - Food Delivery,4.548561573,305034.0,"5,000,000+",Varies with device,0,Everyone,"March 29, 2019",Varies with device,Varies with device,...,0,0,0,0,0,0,0,0,0,0
1,TripAdvisor Hotels Flights Restaurants Attract...,4.400671482,1207922.0,"100,000,000+",Varies with device,0,Everyone,"March 29, 2019",Varies with device,Varies with device,...,0,0,0,0,0,0,0,1,0,0
2,Peapod,3.656329393,1967.0,"100,000+",1.4M,0,Everyone,"September 20, 2018",5.0 and up,2.2.0,...,0,0,0,1,0,0,0,0,0,0
3,foodpanda - Local Food Delivery,4.107232571,389154.0,"10,000,000+",16M,0,Everyone,"March 22, 2019",4.2 and up,4.18.2,...,0,0,0,0,0,0,0,0,0,0
4,My CookBook Pro (Ad Free),4.647752285,2291.0,"10,000+",Varies with device,$5.99,Everyone,"April 1, 2019",Varies with device,Varies with device,...,0,0,0,0,0,0,0,0,0,0


In [146]:
df_dummy['Content Rating'].value_counts(dropna=False)

Everyone           27455
Teen                2860
Everyone 10+        1053
Mature 17+           622
Adults only 18+        3
Unrated                3
Name: Content Rating, dtype: int64

In [174]:
#object(string) values transform to ordinal in Content Rating Feature without nan
df = df_dummy.copy()
df['Content Rating'] = df['Content Rating'].map({'Unrated':0.0,
                                                 'Everyone':1.0,
                                                 'Everyone 10+':2.0,
                                                 'Teen':3.0,
                                                 'Adults only 18+':4.0,
                                                 'Mature 17+':5.0})
df['Content Rating'] = df['Content Rating'].astype(float)
df.head()

,App Name,Rating,Reviews,Installs,Size,Price,Content Rating,Last Updated,Minimum Version,Latest Version,...,Category_PERSONALIZATION,Category_PHOTOGRAPHY,Category_PRODUCTIVITY,Category_SHOPPING,Category_SOCIAL,Category_SPORTS,Category_TOOLS,Category_TRAVEL_AND_LOCAL,Category_VIDEO_PLAYERS,Category_WEATHER
0,DoorDash - Food Delivery,4.548561573,305034.0,"5,000,000+",Varies with device,0,1.0,"March 29, 2019",Varies with device,Varies with device,...,0,0,0,0,0,0,0,0,0,0
1,TripAdvisor Hotels Flights Restaurants Attract...,4.400671482,1207922.0,"100,000,000+",Varies with device,0,1.0,"March 29, 2019",Varies with device,Varies with device,...,0,0,0,0,0,0,0,1,0,0
2,Peapod,3.656329393,1967.0,"100,000+",1.4M,0,1.0,"September 20, 2018",5.0 and up,2.2.0,...,0,0,0,1,0,0,0,0,0,0
3,foodpanda - Local Food Delivery,4.107232571,389154.0,"10,000,000+",16M,0,1.0,"March 22, 2019",4.2 and up,4.18.2,...,0,0,0,0,0,0,0,0,0,0
4,My CookBook Pro (Ad Free),4.647752285,2291.0,"10,000+",Varies with device,$5.99,1.0,"April 1, 2019",Varies with device,Varies with device,...,0,0,0,0,0,0,0,0,0,0


In [175]:
# df1 = df.copy()


In [176]:

lists = []
for i in df["Minimum Version"]:
    if ' and up' in i:
        i = float(i.replace(' and up','')[:3])
        lists.append(i)
    elif '-' in i:
        sep = '-'
        i = float((i.split(sep, 1)[0])[:3])
        lists.append(i)
    else:
        lists.append("Varies with device")
    
k = pd.Series(lists)
median = k[k!="Varies with device"].median()
k = [median if i=="Varies with device" else i for i in k]
df["Minimum Version"] = k

del k, median

In [177]:
df['Latest Version'].unique()

array(['Varies with device', '2.2.0', '4.18.2', ..., ':1.38.3+1',
       '1.0.2274', '1.1.2043'], dtype=object)

In [178]:

lists = []
for i in df["Latest Version"]:
    i = re.sub("[^0-9.]", "", i)
    if '.1.' in i:
        lists.append('-')
    elif '.' in i:
        i = float(i[:3])
        lists.append(i)
    else :
        lists.append("-")

    
k = pd.Series(lists)
median = k[k!="-"].median()
k = [median if i=="-" else i for i in k]
df["Latest Version"] = k



ValueError: could not convert string to float: '.1.'

In [113]:
lists

['-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-'

In [17]:
#change type to float
df2 = df.copy()
df2['Reviews'] = df2['Reviews'].astype(float)

In [18]:
df2["Size"].value_counts()

Varies with device    3195
11M                    859
12M                    711
13M                    616
14M                    598
                      ... 
87k                      1
333k                     1
683k                     1
130k                     1
383k                     1
Name: Size, Length: 576, dtype: int64

In [19]:
#clean 'M','k', fill 'Varies with device' with median and transform to float 
lists = []
for i in df2["Size"]:
    if 'M' in i:
        i = float(i.replace('M',''))
        i = i*1000000
        lists.append(i)
    elif 'k' in i:
        i = float(i.replace('k','').replace(',',''))
        i = i*1000
        lists.append(i)
    else:
        lists.append("Unknown")
    
k = pd.Series(lists)
median = k[k!="Unknown"].median()
k = [median if i=="Unknown" else i for i in k]
df2["Size"] = k

del k,median,lists

In [20]:
#clean '$' and transform to float 
df2['Price'] = [ float(i.split('$')[1]) if '$' in i else float(0) for i in df2['Price'] ] 


In [21]:
print("old: ",df['Price'][9054]," new: ",df2['Price'][9054])

old:  0  new:  0.0


In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31996 entries, 0 to 31999
Data columns (total 59 columns):
App Name                        31996 non-null object
Rating                          31996 non-null object
Reviews                         31996 non-null float64
Installs                        31996 non-null object
Size                            31996 non-null float64
Price                           31996 non-null float64
Content Rating                  31996 non-null float64
Last Updated                    31996 non-null object
Minimum Version                 31996 non-null float64
Latest Version                  31996 non-null object
Category_ART_AND_DESIGN         31996 non-null uint8
Category_AUTO_AND_VEHICLES      31996 non-null uint8
Category_BEAUTY                 31996 non-null uint8
Category_BOOKS_AND_REFERENCE    31996 non-null uint8
Category_BUSINESS               31996 non-null uint8
Category_COMICS                 31996 non-null uint8
Category_COMMUNICATION      

In [23]:
df2.Installs.unique()

array(['5,000,000+', '100,000,000+', '100,000+', '10,000,000+', '10,000+',
       '1,000,000+', '50,000,000+', '500,000+', '50,000+', '5,000+',
       '1,000+', '500,000,000+', '1,000,000,000+', '5,000,000,000+',
       '100+', '500+', '50+', '5+', '10+', '1+', '0+'], dtype=object)

In [24]:
df2["Installs"] = [ float(i.replace('+','').replace(',', '')) if '+' in i or ',' in i else float(0) for i in df2["Installs"] ]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31996 entries, 0 to 31999
Data columns (total 59 columns):
App Name                        31996 non-null object
Rating                          31996 non-null object
Reviews                         31996 non-null float64
Installs                        31996 non-null float64
Size                            31996 non-null float64
Price                           31996 non-null float64
Content Rating                  31996 non-null float64
Last Updated                    31996 non-null object
Minimum Version                 31996 non-null float64
Latest Version                  31996 non-null object
Category_ART_AND_DESIGN         31996 non-null uint8
Category_AUTO_AND_VEHICLES      31996 non-null uint8
Category_BEAUTY                 31996 non-null uint8
Category_BOOKS_AND_REFERENCE    31996 non-null uint8
Category_BUSINESS               31996 non-null uint8
Category_COMICS                 31996 non-null uint8
Category_COMMUNICATION     

In [25]:
df2["Last Updated"][:3]

0        March 29, 2019
1        March 29, 2019
2    September 20, 2018
Name: Last Updated, dtype: object

In [26]:
from datetime import datetime
df3 = df2.copy()
df3["Last Updated"] = [datetime.strptime(i, '%B %d, %Y') for i in df3["Last Updated"]]

In [27]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31996 entries, 0 to 31999
Data columns (total 59 columns):
App Name                        31996 non-null object
Rating                          31996 non-null object
Reviews                         31996 non-null float64
Installs                        31996 non-null float64
Size                            31996 non-null float64
Price                           31996 non-null float64
Content Rating                  31996 non-null float64
Last Updated                    31996 non-null datetime64[ns]
Minimum Version                 31996 non-null float64
Latest Version                  31996 non-null object
Category_ART_AND_DESIGN         31996 non-null uint8
Category_AUTO_AND_VEHICLES      31996 non-null uint8
Category_BEAUTY                 31996 non-null uint8
Category_BOOKS_AND_REFERENCE    31996 non-null uint8
Category_BUSINESS               31996 non-null uint8
Category_COMICS                 31996 non-null uint8
Category_COMMUNICAT

In [28]:
df3 = df3.set_index("Last Updated")
df4 = df3.sort_index()
df4.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31996 entries, 2010-03-26 to 2019-04-04
Data columns (total 58 columns):
App Name                        31996 non-null object
Rating                          31996 non-null object
Reviews                         31996 non-null float64
Installs                        31996 non-null float64
Size                            31996 non-null float64
Price                           31996 non-null float64
Content Rating                  31996 non-null float64
Minimum Version                 31996 non-null float64
Latest Version                  31996 non-null object
Category_ART_AND_DESIGN         31996 non-null uint8
Category_AUTO_AND_VEHICLES      31996 non-null uint8
Category_BEAUTY                 31996 non-null uint8
Category_BOOKS_AND_REFERENCE    31996 non-null uint8
Category_BUSINESS               31996 non-null uint8
Category_COMICS                 31996 non-null uint8
Category_COMMUNICATION          31996 non-null uint8
Category_DA

In [29]:
df4.isna().any().sum()

0

In [30]:
data = df4.copy()
data.shape

(31996, 58)

In [31]:
data.to_csv('preprocessed.csv')

In [32]:
data['Content Rating']

Last Updated
2010-03-26    3.0
2010-05-02    1.0
2010-05-23    1.0
2010-07-07    1.0
2010-09-06    1.0
             ... 
2019-04-04    1.0
2019-04-04    1.0
2019-04-04    1.0
2019-04-04    1.0
2019-04-04    3.0
Name: Content Rating, Length: 31996, dtype: float64

In [40]:
data['Latest Version'].unique()

array(['1.0.2', '1.3', '0.9.12', ..., '3.75', '4.2.30g', '1.0.3.4'],
      dtype=object)

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31996 entries, 2010-03-26 to 2019-04-04
Data columns (total 58 columns):
App Name                        31996 non-null object
Rating                          31996 non-null object
Reviews                         31996 non-null float64
Installs                        31996 non-null float64
Size                            31996 non-null float64
Price                           31996 non-null float64
Content Rating                  31996 non-null float64
Minimum Version                 31996 non-null float64
Latest Version                  31996 non-null object
Category_ART_AND_DESIGN         31996 non-null uint8
Category_AUTO_AND_VEHICLES      31996 non-null uint8
Category_BEAUTY                 31996 non-null uint8
Category_BOOKS_AND_REFERENCE    31996 non-null uint8
Category_BUSINESS               31996 non-null uint8
Category_COMICS                 31996 non-null uint8
Category_COMMUNICATION          31996 non-null uint8
Category_DA